In [1]:
cd Desktop/BIA-660/Final Project/data

/Users/linzeyang/Desktop/BIA-660/Final Project/data


In [54]:
import pandas as pd
import numpy as np
import re
import datetime
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from six.moves import urllib
import json
import collections
import itertools
import requests
import time
import pandas as pd
from  __builtin__ import any as b_any
import math
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
%matplotlib inline


/Users/linzeyang/anaconda/envs/py2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [31]:
import pandas as pd
df = pd.read_csv('201611-citibike-tripdata.csv')

In [32]:
df.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,331,2016-11-01 00:00:08,2016-11-01 00:05:40,324,DeKalb Ave & Hudson Ave,40.689888,-73.981013,419,Carlton Ave & Park Ave,40.695807,-73.973556,21959,Subscriber,1981.0,1
1,740,2016-11-01 00:00:08,2016-11-01 00:12:29,161,LaGuardia Pl & W 3 St,40.729170,-73.998102,265,Stanton St & Chrystie St,40.722293,-73.991475,20172,Subscriber,1991.0,1
2,229,2016-11-01 00:00:11,2016-11-01 00:04:00,3074,Montrose Ave & Bushwick Ave,40.707678,-73.940162,3068,Humboldt St & Varet St,40.703172,-73.940636,22995,Subscriber,1984.0,1
3,155,2016-11-01 00:00:18,2016-11-01 00:02:53,3357,W 106 St & Amsterdam Ave,40.800836,-73.966449,3323,W 106 St & Central Park West,40.798186,-73.960591,22022,Subscriber,1973.0,1
4,772,2016-11-01 00:00:23,2016-11-01 00:13:15,346,Bank St & Hudson St,40.736529,-74.006180,432,E 7 St & Avenue A,40.726218,-73.983799,17324,Subscriber,1988.0,1


In [33]:
# missing values in columns
df.isnull().sum()

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                  12534
Birth Year                 96366
Gender                         0
dtype: int64

In [34]:
# Snake_case the columns
import re
def camel_to_snake(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).replace(' ','').lower()

df.columns = [camel_to_snake(col) for col in df.columns]
df.head()

,trip_duration,start_time,stop_time,start_stationid,start_station_name,start_station_latitude,start_station_longitude,end_stationid,end_station_name,end_station_latitude,end_station_longitude,bikeid,user_type,birth_year,gender
0,331,2016-11-01 00:00:08,2016-11-01 00:05:40,324,DeKalb Ave & Hudson Ave,40.689888,-73.981013,419,Carlton Ave & Park Ave,40.695807,-73.973556,21959,Subscriber,1981.0,1
1,740,2016-11-01 00:00:08,2016-11-01 00:12:29,161,LaGuardia Pl & W 3 St,40.729170,-73.998102,265,Stanton St & Chrystie St,40.722293,-73.991475,20172,Subscriber,1991.0,1
2,229,2016-11-01 00:00:11,2016-11-01 00:04:00,3074,Montrose Ave & Bushwick Ave,40.707678,-73.940162,3068,Humboldt St & Varet St,40.703172,-73.940636,22995,Subscriber,1984.0,1
3,155,2016-11-01 00:00:18,2016-11-01 00:02:53,3357,W 106 St & Amsterdam Ave,40.800836,-73.966449,3323,W 106 St & Central Park West,40.798186,-73.960591,22022,Subscriber,1973.0,1
4,772,2016-11-01 00:00:23,2016-11-01 00:13:15,346,Bank St & Hudson St,40.736529,-74.006180,432,E 7 St & Avenue A,40.726218,-73.983799,17324,Subscriber,1988.0,1


In [35]:
# parse start_time and stop_time
import datetime

try:
    df['start_datetime'] = [datetime.datetime.strptime(x, '%m/%d/%Y %H:%M:%S') for x in df.start_time]
except ValueError:
    df['start_datetime'] = [datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in df.start_time]
df['start_day'] = [x.strftime('%Y-%m-%d') for x in df.start_datetime]

df.head()

,trip_duration,start_time,stop_time,start_stationid,start_station_name,start_station_latitude,start_station_longitude,end_stationid,end_station_name,end_station_latitude,end_station_longitude,bikeid,user_type,birth_year,gender,start_datetime,start_day
0,331,2016-11-01 00:00:08,2016-11-01 00:05:40,324,DeKalb Ave & Hudson Ave,40.689888,-73.981013,419,Carlton Ave & Park Ave,40.695807,-73.973556,21959,Subscriber,1981.0,1,2016-11-01 00:00:08,2016-11-01
1,740,2016-11-01 00:00:08,2016-11-01 00:12:29,161,LaGuardia Pl & W 3 St,40.729170,-73.998102,265,Stanton St & Chrystie St,40.722293,-73.991475,20172,Subscriber,1991.0,1,2016-11-01 00:00:08,2016-11-01
2,229,2016-11-01 00:00:11,2016-11-01 00:04:00,3074,Montrose Ave & Bushwick Ave,40.707678,-73.940162,3068,Humboldt St & Varet St,40.703172,-73.940636,22995,Subscriber,1984.0,1,2016-11-01 00:00:11,2016-11-01
3,155,2016-11-01 00:00:18,2016-11-01 00:02:53,3357,W 106 St & Amsterdam Ave,40.800836,-73.966449,3323,W 106 St & Central Park West,40.798186,-73.960591,22022,Subscriber,1973.0,1,2016-11-01 00:00:18,2016-11-01
4,772,2016-11-01 00:00:23,2016-11-01 00:13:15,346,Bank St & Hudson St,40.736529,-74.006180,432,E 7 St & Avenue A,40.726218,-73.983799,17324,Subscriber,1988.0,1,2016-11-01 00:00:23,2016-11-01


## group by date and station

In [36]:
df['bikein']=1
df['bikeout']=1

df_bikeout = df.groupby(by=['start_stationid','start_day'])['bikeout'].count()
df_bikein = df.groupby(by=['end_stationid','start_day'])['bikein'].count()

In [37]:
df_bikeout = pd.DataFrame(df_bikeout)

In [38]:
df_out = df_bikeout.reset_index(drop = False)
df_out.rename_axis({'start_stationid':'station_id'}, axis='columns', inplace=True)


In [39]:
df_bikein = pd.DataFrame(df_bikein)

In [40]:
df_in = df_bikein.reset_index(drop = False)
df_in.rename_axis({'end_stationid':'station_id'}, axis='columns', inplace=True)

In [41]:
df_dayloss = df_out.merge(right = df_in, left_on=['station_id','start_day'], right_on=['station_id','start_day'], how='outer')
# or join
# df_out.join(df_in.set_index(['station_id','start_day']), on=['station_id','start_day'], how='outer')
df_dayloss = df_dayloss.sort_values(['station_id', 'start_day'])

In [42]:
len(df_dayloss)

17779

In [55]:

# =========== count docks Then append to dayloss============
jsonurl = urllib.request.urlopen('https://gbfs.citibikenyc.com/gbfs/en/station_status.json')
text = json.loads(jsonurl.read()) # <-- read from it
data=text['data']
#print(len(data['stations']))   664 stations
#print(data['stations'][0]['station_id']) # station id

# station id Extract:
total_stations_id = []
for i in range(len(data['stations'])):
    id=data['stations'][i]['station_id']
    total_stations_id.append(id)
total_stations_id=[int(x) for x in total_stations_id]
#print('station_id list:'+str(total_stations_id))

# num_bikes_available Extract:
total_num_bikes_available = []
for i in range(len(data['stations'])):
    num_bikes_available = data['stations'][i]['num_bikes_available']
    total_num_bikes_available.append(num_bikes_available)
#print('bikes availble:'+str(total_num_bikes_available))

# num_docks_available Extract:
total_docks_available = []
for i in range(len(data['stations'])):
    docks_availble=data['stations'][i]['num_docks_available']
    total_docks_available.append(docks_availble)
#print('docks availble:'+str(total_docks_available))

total_docks=[x + y for x, y in zip(total_num_bikes_available, total_docks_available)]
#print('total docks for each station:'+str(total_docks))
#to_dict

new_dict = dict(zip(total_stations_id,total_docks))
#print('zip station_id and total docks:'+str(new_dict))

# append docks to df_dayloss
df_dayloss['docks'] = df_dayloss['station_id'].map(new_dict)


## bike loss per day for each station

In [57]:
df_dayloss = df_dayloss.fillna(0)

In [58]:
df_dayloss['bike_loss'] = df_dayloss['bikeout'] - df_dayloss['bikein']

In [59]:
%%time
df_dayloss0 = df_dayloss
for i in df_dayloss0.index.values:
    try:
        df_dayloss0.loc[i, 'latitude'] = df.loc[df.start_stationid == df_dayloss.loc[i, 'station_id'], 'start_station_latitude'].unique()[0]
        df_dayloss0.loc[i, 'longitude'] = df.loc[df.start_stationid == df_dayloss.loc[i, 'station_id'], 'start_station_longitude'].unique()[0]
    except:
        print i
        continue

17727
17728
17729
17730
17731
17732
17733
17734
17747
17748
17749
17750
17751
17752
17753
17754
17755
17765
17778
CPU times: user 1min 40s, sys: 1.14 s, total: 1min 41s
Wall time: 1min 43s


In [62]:
bike_left=df_dayloss0.dropna()

In [77]:
bike_left['docks']=bike_left['docks'].replace(0,30)

In [78]:
bike_left['75%docks']=(0.75*bike_left['docks'])
bike_left['left_bike']=0

In [79]:
np.mean(bike_left['docks'])

30.980651330221047

In [80]:
for i in range(len(bike_left['bike_loss'])):
    if bike_left['75%docks'][i]-bike_left['bike_loss'][i] < bike_left['docks'][i]:
        bike_left['left_bike'][i]=bike_left['75%docks'][i]-bike_left['bike_loss'][i]
    else:
        bike_left['left_bike'][i]=bike_left['docks'][i]

/Users/linzeyang/anaconda/envs/py2.7/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/linzeyang/anaconda/envs/py2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [81]:
bike_left

,station_id,start_day,bikeout,bikein,bike_loss,latitude,longitude,docks,75%docks,left_bike
0,72,2016-11-01,133.0,127.0,6.0,40.767272,-73.993929,37.0,27.75,21
1,72,2016-11-02,119.0,130.0,-11.0,40.767272,-73.993929,37.0,27.75,37
2,72,2016-11-03,146.0,144.0,2.0,40.767272,-73.993929,37.0,27.75,25
3,72,2016-11-04,129.0,108.0,21.0,40.767272,-73.993929,37.0,27.75,6
4,72,2016-11-05,92.0,109.0,-17.0,40.767272,-73.993929,37.0,27.75,37
5,72,2016-11-06,96.0,90.0,6.0,40.767272,-73.993929,37.0,27.75,21
6,72,2016-11-07,116.0,105.0,11.0,40.767272,-73.993929,37.0,27.75,16
7,72,2016-11-08,128.0,137.0,-9.0,40.767272,-73.993929,37.0,27.75,36
8,72,2016-11-09,87.0,80.0,7.0,40.767272,-73.993929,37.0,27.75,20
9,72,2016-11-10,128.0,117.0,11.0,40.767272,-73.993929,37.0,27.75,16


In [83]:
bike_left.to_csv('bike_left11_train.csv')